# Playing with learning rate
---
## Visualization of the effects of wrong choice of learning rate 

***Author: Piotr Skalski***

### Imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

### Settings

In [2]:
# parameters a and b of the real function
REAL_PARAMS = [1, 1]
# starting point for gradient descent
INIT_PARAMS = [-0.5, -1]
# output directory (the folder must be created on the drive)
OUTPUT_DIR = "playing_with_learning_rate"

### Performing the simulation

In [3]:
def find_optimization_path(tf_function, init_point, iterations, learning_rate):
    x, y = [tf.Variable(initial_value=p, dtype=tf.float32) for p in init_point]
    function = tf_function(x, y)
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(function)
    x_list, y_list, cost_list = [], [], []
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for t in range(iterations):
            x_, y_, function_ = sess.run([x, y, function])
            x_list.append(x_); y_list.append(y_); cost_list.append(function_)
            result, _ = sess.run([function, train_op])
            
    return x_list, y_list, cost_list

### Create a blank chart

In [4]:
def create_blank_chart_with_styling(plot_size):
    # my favorite styling kit
    plt.style.use('dark_background')
    # determining the size of the graph
    fig = plt.figure(figsize=plot_size)    
    # 3D mode
    ax = Axes3D(fig)
    # transparent axis pane background 
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    # setting chart axis names
    ax.set(xlabel="$x$", ylabel="$y$")
    return (fig, ax)

### Create animation

In [5]:
def create_animation(tf_function, np_function, init_point, iterations, learning_rate, plot_name, file_name, dir_name):
    # 3D cost figure
    for angle in range(iterations):
        fix, ax = create_blank_chart_with_styling((6, 6))
        x_list, y_list, cost_list = find_optimization_path(tf_function, init_point, iterations, learning_rate)
        # parameter space
        a3D, b3D = np.meshgrid(np.linspace(-4, 4, 100), np.linspace(-4, 4, 100))
        cost3D = np.array([np_function(x_, y_) for x_, y_ in zip(a3D.flatten(), b3D.flatten())]).reshape(a3D.shape)
        ax.plot_surface(a3D, b3D, cost3D, rstride=1, cstride=1, cmap=plt.get_cmap('rainbow'), alpha=1.0)
        # plot 3D gradient descent
        if angle < 10:
            ax.plot(x_list[:angle], y_list[:angle], zs=cost_list[:angle], zdir='z', c='r', lw=2)
        else:
            ax.plot(x_list[angle-10:angle], y_list[angle-10:angle], zs=cost_list[angle-10:angle], zdir='z', c='r', lw=2)
        # graph rotation
        ax.view_init(30, 225 + angle*2)
        # addition of a title
        ax.set_title(plot_name, fontsize=20)
        # saving a file
        plt.savefig("./{}/{}_{:05}.png".format(dir_name, file_name, angle))
        plt.close()

In [6]:
tf_fun = lambda x, y: 3*(1-x)**2*tf.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5)*tf.exp(-x**2-y**2) - 1/3*tf.exp(-(x+1)**2 - y**2)   
np_fun = lambda x, y: 3*(1-x)**2*np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5)*np.exp(-x**2-y**2) - 1/3*np.exp(-(x+1)**2 - y**2)   

create_animation(tf_fun, np_fun, INIT_PARAMS, 180, 0.15, "Big learning rate", "big_rate", OUTPUT_DIR)
create_animation(tf_fun, np_fun, INIT_PARAMS, 180, 0.01, "Small learning rate", "small_rate", OUTPUT_DIR)

### Expected results

Go to OUTPUT_DIR, which should now be filled with subsequent keyframes of our animation. All the resulting images look more or less like this.

Now all you need to do is enter OUTPUT_DIR and use ImageMagick to create a final gift with one command.

```bash
convert -delay 10 -loop 0 *.png keras_class_boundaries.gif
```

## Thank you
---